# Test pubsub in Redis and send data to Fluent

In [6]:
import redis
import time

from fluent import sender

# Logging with Fluent
fluentHost = 'fluent-server'
#fluentHost = 'localhost'
logger = sender.FluentSender('python-client', host=fluentHost, port=24224)
logger.emit('init', {'val': 43})


# REDIS
# When using docker compose
redisHost = 'redis-server'
# Connect
r = redis.Redis(host=redisHost, port=6379, db=0)

# Listen to messages
p = r.pubsub()

def custom_handler(message):
	# do_something with the message
	print(message)
	# Send a copy to log
	logger.emit('message', message)

# listen to all chanels
p.psubscribe(**{'*':custom_handler})

# Run in a thread for now
thread = p.run_in_thread(sleep_time=0.001)
